Investigating whether a random projection would inhibit the ability to transfer across tasks.

Testing both modular and monolithic networks.

KMNIST
Procedures:

    - initialize training on 2 tasks
    - Train on one task.
    - Train on another identical task (but with different random projection).

In [1]:
from shell.datasets.datasets import get_dataset
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from shell.utils.utils import seed_everything, viz_embedding
import torch
import subprocess
import torch.nn as nn
import torch
import os
from omegaconf import DictConfig
from shell.datasets.datasets import get_dataset
from shell.utils.utils import seed_everything
from pprint import pprint
from shell.fleet.network import TopologyGenerator
from shell.models.cnn_soft_lifelong_dynamic import CNNSoftLLDynamic
from shell.models.cnn import CNN
from shell.models.mlp import MLP
from shell.models.mlp_soft_lifelong_dynamic import MLPSoftLLDynamic
from shell.learners.er_dynamic import CompositionalDynamicER
from shell.learners.er_nocomponents import NoComponentsER
from shell.utils.experiment_utils import eval_net
from shell.utils.experiment_utils import setup_experiment
from sklearn.manifold import TSNE
import logging
import seaborn as sns
from shell.fleet.data_fleet import least_confidence_scorer, entropy_scorer, margin_scorer, cross_entropy_scorer
logging.basicConfig(level=logging.INFO)

In [2]:
data_cfg = {
    "dataset_name": "kmnist",
    "num_tasks": 4,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    "labels": np.array([1, 2, 3, 4, 5, 6, 5, 6]),
    'remap_labels': True,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [1 2 3 4 5 6 5 6]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)


In [3]:
net_cfg = {
    'depth': 2,
    'layer_size': 64,
    'num_init_tasks': -1,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': 1,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': True,
}
net = MLP(**net_cfg)
agent = NoComponentsER(net, **agent_cfg)


In [4]:
for task_id in range(2):
    testloaders = {task: torch.utils.data.DataLoader(testset,
                                                 batch_size=128,
                                                 shuffle=False,
                                                 num_workers=0,
                                                 pin_memory=True,
                                                 ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}

    trainloader = torch.utils.data.DataLoader(dataset.trainset[task_id],
                                            batch_size=64,
                                            shuffle=True,
                                            num_workers=0,
                                            pin_memory=True,
                                            )
    agent.train(trainloader, task_id=0, num_epochs=100, testloaders=testloaders,
                    save_freq=1,
                    train_mode='both')

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 6.309	acc: 0.500
INFO:root:	task: avg	loss: 6.309	acc: 0.500
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 6.305	acc: 0.500
INFO:root:	task: avg	loss: 6.305	acc: 0.500


IndexError: index 1 is out of range